In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
data = {'ID': [],
        'text': [],
        'prediction' : []}

In [3]:
test_data = {'ID': [],
        'text': [],
        'prediction' : []}

In [4]:
x_train = []
y_train = []

x_test = []
y_test = []
y_test_id = []

y_train_cc = []
y_train_cd = []

x_train_cc_ids = []
x_train_cd_ids = []

Ανάγνωση των δεδομένων εκπαίδευσης και δημιουργία dataset. Κατά την διάρκεια της διαπέρασης των δεδομένων, διατηρούνται μόνο τα δεδομένα ομιλίας του συμμετέχοντα, ενώ κάποια χαρακτηριστικά της κωδικοποίησης CHAT αντικαθίστανται με σύμβολα

------------------------------------------------------------

Reading training data and creating the dataset. While reading the data, only the speech of each participant is kept, while some of CHAT encoding's characteristics are replaced by symbols

In [5]:
import glob
import re

#train data for non-AD
path = glob.glob("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription"+"/*/*.cha")

for filename in path:
  # print("-------------------------------------", filename, "-------------------------------------")
  train_file = open(filename, "r")
  # print(filename)

  keep_filename = filename
  keep_filename = keep_filename.replace("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cc/", "")
  keep_filename = keep_filename.replace("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/", "")
  keep_filename = keep_filename.replace(".cha", "")
  # print(keep_filename)
  x_train_cc_ids.append(keep_filename)

  train_text_non_AD = train_file.read()
  # x_train.append(train_text_non_AD)
  cc_or_cd = filename.replace("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/", "")
  if "cc" in cc_or_cd:
    y_train_cc.append(0)
  else :
    y_train_cc.append(1)


  sentences = train_text_non_AD.split("\n")
  regex = "[^a-zA-Z']+"
  p=re.compile(regex)
   #this is so that i can seperate the words and take out the symbols

  par_talk_sentences = ""
  begin_answer = False
  for item in sentences:
    # print(item)
    if item.startswith('*PAR'):
      # print(item)
      par_talk_sentences += item
      begin_answer = True #check if a *PAR has started in the previous line

    elif begin_answer == True:
      if not item.startswith('*') and not item.startswith('%') and not item.startswith('@'): #if it has started but has one of those symbol, it ends (else)
        # print(item)
        par_talk_sentences += item
        # if(re.search(p, item)):
        #   print(item)
      else:
         begin_answer = False

    elif (begin_answer==False):
      pass

  stupid_counter = 0
  str_all_sentences = ""
  par_talk_words = par_talk_sentences.split()
  keep_words = []
  begin_x = False
  for item in par_talk_words:
    # print(item)
    str_all_sentences += item
    if(not re.search(p, item)):
      if(not (item == "xxx" or item == "xx")):
        keep_words.append(item)
      else:
        keep_words.append(".")
    elif "(" in item:
      new_word = item.replace("(", "")
      new_word = new_word.replace(")", "")
      if(not re.search(p, new_word)):
        new_word += " - "
        keep_words.append(new_word)
    elif("&" in item):
      item = item.replace("&", "")
      item = item + " . "
      if(not re.search(p, item)):
        keep_words.append(item)
    elif "[/]" in item:
      keep_words.append(".")
    elif "[//]" in item:
      keep_words.append("..")
    elif "+//" in item:
      keep_words.append("...")
    elif "[:" in item:
      item = item.replace("[:", "")
      item = item.replace("]", "")
      item = item.replace("+", "")
      keep_words.append(item)



  words_in_str = ""
  for item in keep_words:
    words_in_str += item
    words_in_str += " "


  data['ID'].append(keep_filename)
  data['text'].append(words_in_str)
  if "cc" in cc_or_cd:
    data['prediction'].append(0)
  else :
    data['prediction'].append(1)

  # print(keep_words)


In [6]:
import pandas as pd

# print(data)

df = pd.DataFrame(data)
print(df.columns)
# print(df)

Index(['ID', 'text', 'prediction'], dtype='object')


Ανάγνωση των δεδομένων ελέγχου και δημιουργία dataset. Κατά την διάρκεια της διαπέρασης των δεδομένων, διατηρούνται μόνο τα δεδομένα ομιλίας του συμμετέχοντα, ενώ κάποια χαρακτηριστικά της κωδικοποίησης CHAT αντικαθίστανται με σύμβολα. Η μόνη διαφορά με την ανάγνωση των δεδομένων εκπαίδευσης είναι ότι το prediction των δεδομένων ελέγχου θα αξιοποιηθούν παρακάτω, από το αρχείο labels.txt

------------------------------------------------------------

Reading training data and creating the dataset. While reading the data, only the speech of each participant is kept, while some of CHAT encoding's characteristics are replaced by symbols. The only difference between reading the train and test data, is that the predictions of the test data will be used later, from the file labels.txt

In [7]:
#test data
path = glob.glob("/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/transcription"+"/*.cha")
words_per_participant_test = []

for filename in path:
  # print("-------------------------------------", filename, "-------------------------------------")
  test_file = open(filename, "r")
  keep_filename = filename
  keep_filename = keep_filename.replace("/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/transcription/", "")
  keep_filename = keep_filename.replace(".cha", "")
  # print(keep_filename)
  # print(filename)
  test_text_all = test_file.read()
  # print(test_text_all)

  sentences = test_text_all.split("\n")
  per_talk_sentences_test = ""
  begin_answer = False
  for item in sentences:
    if item.startswith('*PAR'):
      per_talk_sentences_test += item
      begin_answer = True

    elif begin_answer == True:
      if not item.startswith('*') and not item.startswith('%') and not item.startswith('@'):
        per_talk_sentences_test += item

      else:
        begin_answer = False

    elif begin_answer == False:
      pass

  str_all_sentences_test = ""
  par_talk_words_test = per_talk_sentences_test.split()
  keep_words = []
  for item in par_talk_words_test:
    str_all_sentences_test += item

    if not re.search(p, item):
      if(not (item == "xxx" or item == "xx")):
        keep_words.append(item)
      else:
        keep_words.append(".")
    elif "(" in item:
      new_word = item.replace("(", "")
      new_word = new_word.replace(")", "")
      if(not re.search(p, new_word)):
        new_word += " - "
        keep_words.append(new_word)
    else:
      if "&" in item:
        item = item.replace("&", "")
        item = item + " . "
        if(not re.search(p, item)):
          keep_words.append(item)
      elif "[/]" in item:
        keep_words.append(".")
      elif "[//]" in item:
        keep_words.append("..")
      elif "+//" in item:
        keep_words.append("...")
      elif "[:" in item:
        item = item.replace("[:", "")
        item = item.replace("]", "")
        item = item.replace("+", "")
        keep_words.append(item)

  words_in_str = ""
  for item in keep_words:
    words_in_str += item
    words_in_str += " "

  test_data['ID'].append(keep_filename)
  test_data['text'].append(words_in_str)
  test_data['prediction'].append(-1)


In [8]:
# print(test_data)

df_t = pd.DataFrame(test_data)
print(df_t.columns)
# print(df_t)

Index(['ID', 'text', 'prediction'], dtype='object')


Διάβασμα των predictions και ενσωματωση τους με το υπόλοιπο dataset

----------------------------------------------------------------------

Reading the predictions and adjusting them to the whole dataset

In [9]:
#na paro to y_test
filename = "/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/labels.txt"
test_file = open(filename, "r")
# print(filename)
test_text_all = test_file.read()

index_id = -1
index_label = -1
age = -1

sentences = test_text_all.split("\n")
first_line_words = sentences[0].split(';')
i=0
for label in first_line_words:
  # print(label)
  label = label.replace(" ", "")
  # print(label)
  if label == "ID":
    index_id = i
    # print("id in label: ", i)
  elif label == "Label":
    index_label = i
    # print("label in label: ", i)
  elif label == "age":
    age = i

  i += 1

new_sentences = []
for sentence in sentences:
  new_sentence = []
  sentence = sentence.split(";")
  print(" s: ", sentence)
  for i in sentence:
    i = i.replace(" ", "")
    new_sentence.append(i)

  new_sentences.append(new_sentence)

i=0
y_test_id = []
ages = []
for sentence in new_sentences:
  # print ("sentence: ", sentence)

  if i != 0:
    y_test_id.append(sentence[index_id])
    y_test.append(int(sentence[index_label]))
    ages.append(sentence[age])
  i+= 1

# print("_________________________________________________")
# print(y_test_id)
# print(len(new_sentences))
# print("len age: ", len(ages))
# print(len(y_test_id))


i=0
for i in range(len(y_test_id)):
  list_obj = [y_test_id[i]]
  df_t.loc[df_t['ID'].isin(list_obj), 'prediction'] = y_test[i]



 s:  ['ID   ', ' age', ' gender', 'Label ', 'mmse']
 s:  ['S160 ', ' 63', ' 1 ', ' 0 ', ' 28']
 s:  ['S161 ', ' 55', ' 1 ', ' 0 ', ' 29']
 s:  ['S162 ', ' 67', ' 1 ', ' 1 ', ' 24']
 s:  ['S163 ', ' 71', ' 0 ', ' 0 ', ' 30']
 s:  ['S164 ', ' 73', ' 1 ', ' 1 ', ' 21']
 s:  ['S165 ', ' 64', ' 0 ', ' 1 ', ' 15']
 s:  ['S166 ', ' 73', ' 1 ', ' 0 ', ' 29']
 s:  ['S167 ', ' 65', ' 0 ', ' 1 ', ' 28']
 s:  ['S168 ', ' 71', ' 0 ', ' 1 ', ' 27']
 s:  ['S169 ', ' 78', ' 0 ', ' 1 ', ' 26']
 s:  ['S170 ', ' 51', ' 0 ', ' 0 ', ' 28']
 s:  ['S171 ', ' 69', ' 1 ', ' 1 ', ' 23']
 s:  ['S172 ', ' 78', ' 1 ', ' 0 ', ' 30']
 s:  ['S173 ', ' 79', ' 1 ', ' 1 ', ' 17']
 s:  ['S174 ', ' 67', ' 1 ', ' 0 ', ' 29']
 s:  ['S175 ', ' 69', ' 1 ', ' 0 ', ' 30']
 s:  ['S176 ', ' 69', ' 0 ', ' 1 ', ' 27']
 s:  ['S177 ', ' 65', ' 0 ', ' 0 ', ' 30']
 s:  ['S178 ', ' 68', ' 1 ', ' 0 ', ' 30']
 s:  ['S179 ', ' 74', ' 0 ', ' 1 ', ' 10']
 s:  ['S180 ', ' 74', ' 0 ', ' 0 ', ' 29']
 s:  ['S181 ', ' 65', ' 1 ', ' 1 ', ' 17']
 s

In [10]:
df_t

,ID,text,prediction
0,S172,well the little girl is reaching for a cookie ...,0
1,S171,it's a family a mother and the children and th...,1
2,S174,I see a little boy on a stool almost falling t...,0
3,S166,oh it's the same picture you should give me a ...,0
4,S160,this boy is about to fall off of - the stool ...,0
5,S161,a boy with a cookie in his one hand and his ha...,0
6,S167,the pearl mom's getting - her wet .. feet we...,1
7,S170,I see this woman who's standing - by the sink...,0
8,S168,well the . the little boy is stealing cookies ...,1
9,S165,yeah I see the woman's in a kitchen and now it...,1


In [11]:
df.to_csv("train_bert_new_4.csv",index=False)
df_t.to_csv("test_bert_new_4.csv",index=False)